In [ ]:
%load_ext autoreload

%autoreload 2

In [ ]:
from environment import *

from setting import *

In [ ]:
path_dict = path(
    NAME,
    ELEMENTS,
    NMF_K,
    WT_HCC_K,
    H_HCC_K,
    UPLOAD_TO_PLOTLY,
)

In [ ]:
feature_x_sample = pd.read_table(
    path_dict['feature_x_sample_file_path'],
    index_col=0,
)

In [ ]:
for element in ELEMENTS:
    
    if element == 'feature':
        
        df = feature_x_sample
        
        to_peek = FEATURES_TO_PEEK
        
    elif element == 'sample':
        
        df = feature_x_sample.T
        
        to_peek = SAMPLES_TO_PEEK
        
    skew_t_pdf_fit_parameter = pd.read_table(
        path_dict['{}_skew_t_pdf_fit_parameter_file_path'.format(element)],
        index_col=0,
    )
    
    context_matrix = ccal.make_context_matrix(
        df,
        n_job=MAX_N_JOB,
        skew_t_pdf_fit_parameter=skew_t_pdf_fit_parameter,
        directory_path=path_dict['{}_directory_path'.format(element)], 
    )
    
    if context_matrix.size < 1e6:
        
        ccal.plot_heat_map(
            context_matrix,
            title='({}) Context'.format(element),
        )
        
    for select_context in (
        'negative',
        'positive',
    ):
        
        if select_context == 'negative':
            
            element_context = context_matrix.clip(upper=0).sum(axis=1)
            
        elif select_context == 'positive':
            
            element_context = context_matrix.clip(lower=0).sum(axis=1)
            
        element_context.sort_values(inplace=True)
        
        if to_peek is None:
            
            ranks = []
            
        else:
            
            ranks = np.nonzero([any(peek == index for peek in to_peek) for index in element_context.index])[0]
            
        title = '({}) {} Context Summary'.format(
            element,
            select_context,
        )
        
        ccal.plot_points(
            (
                tuple(range(element_context.size)),
                ranks,
            ),
            (
                element_context,
                element_context[ranks],
            ),
            names=(
                'All',
                'To Peek',
            ),
            texts=(
                element_context.index,
                element_context.index[ranks],
            ),
            modes=(
                'markers',
                'markers+text',
            ),
            title=title,
            xaxis_title='Rank',
            yaxis_title=title,
        )
        
        n_extreme = 3
        
        if select_context == 'negative':
            
            ranks = range(n_extreme)
            
        else:
            
            ranks = range(
                -n_extreme,
                0,
            )
            
        for rank in ranks:
            
            index = element_context.index[rank]
            
            series = df.loc[index]
            
            n_data, location, scale, degree_of_freedom, shape = skew_t_pdf_fit_parameter.loc[index, [
                'N Data',
                'Location',
                'Scale',
                'Degree of Freedom',
                'Shape',
            ]]
            
            ccal.plot_context(
                series,
                n_data=n_data,
                location=location,
                scale=scale,
                degree_of_freedom=degree_of_freedom,
                shape=shape,
                plot_rug=series.dropna().size < 1e3,
                title='({}) Context Summary Rank {} [{}]'.format(
                    element,
                    rank,
                    index,
                ),
            )